# タスク 1b: コンテキストを含むプロンプトを使用してテキスト生成を実行する

このノートブックでは、カスタマーサポートエンジニアから受けたカスタマーサービスの品質に満足していない顧客へのメールレスポンスを生成する方法を学習します。不満を持った顧客から実際に受け取ったメールの内容を含めることで、モデルに追加のコンテキストを提供します。

同様のユースケースで LangChain フレームワークを活用するために、PromptTemplates の助けを借りて複雑さをさらに追加します。PromptTemplates を使用すると、後で情報を入力してさまざまなシナリオに基づいてモデル出力を取得できる汎用シェルを作成できます。

[LangChain](https://python.langchain.com/docs/get_started/introduction.html) は、言語モデルを利用したアプリケーションを開発するためのフレームワークです。このフレームワークの主要な側面により、さまざまなコンポーネントを連結して高度なユース ケースを作成することで、大規模言語モデルを拡張できます。

プロンプトにコンテキストが追加されているため、このノートブックで生成されるコンテンツは、以前にゼロショットプロンプトで生成されたコンテンツよりも品質と関連性がはるかに高くなります。このノートブックで使用されるプロンプトは、テキスト生成リクエストにコンテキストを追加するためのカスタム LangChain プロンプトテンプレートを作成します。



#### シナリオ
あなたは AnyCompany のカスタマーサービスマネージャーである Bob です。一部の顧客はカスタマーサービスに満足しておらず、カスタマーサポートエンジニアが提供するサービスについて否定的なフィードバックを提供しています。今、あなたはこれらの顧客に謙虚に対応し、質の悪いサービスを謝罪して信頼を取り戻したいと考えています。大量のメールを生成するには、大規模言語モデル (LLM) の助けが必要です。これらのメールは、以前のメールのやり取りから、人間に優しく、顧客の感情に合わせてパーソナライズされています。

このシナリオでは、LangChain の PromptTemplates の機能を活用して、顧客の以前のメールに基づいてパーソナライズされたメール応答を生成するための汎用シェルを作成できます。PromptTemplate は顧客の元のメール コンテンツを組み込み、LLM がコンテキストと感情を理解して、関連性のあるカスタマイズされた応答を生成できるようにします。

## Task 1b.1: 環境のセットアップ

このタスクでは、環境をセットアップします。

In [ ]:
#Create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))


bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))

## タスク 1b.2: Bedrock LLM モデルの呼び出し

このタスクでは、llms から Bedrock クラスのインスタンスを作成します。これには、Amazon Bedrock で使用可能なモデルの Amazon リソース名 (ARN) である `model_id` が必要です。

オプションで、以前に作成した boto3 クライアントと、`temperature`、`top_p`、`max_token_count`、または `stop_sequences` などのパラメータを保持できるいくつかの `model_kwargs` を渡すこともできます (パラメータの詳細については、Amazon Bedrock コンソールで調べることができます)。

Amazon Bedrock で使用可能なテキスト生成モデル ID については、[ドキュメント](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids-arns.html) を参照してください。

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** モデルによってサポートされる `model_kwargs` は異なります。

In [ ]:
# Model configuration
from langchain_aws import ChatBedrock
from langchain_core.output_parsers import StrOutputParser

model_id = "amazon.nova-lite-v1:0"
model_kwargs =  { 
        "max_tokens": 512,
        "temperature": 0,
        "top_p": 1,
}

# LangChain class for chat
chat_model = ChatBedrock(
    client=bedrock_client,
    model_id=model_id,
    model_kwargs=model_kwargs,
)

## Task 1b.3: LangChain カスタム プロンプト テンプレートを作成する

このタスクでは、実行ごとに異なる入力変数を渡すことができるプロンプトのテンプレートを作成します。これは、データベースから取得する可能性のある異なる入力変数を使用してコンテンツを生成する必要がある場合に便利です。

前のタスクでは、プロンプトをハードコードしました。複数の顧客が同様の否定的なフィードバックを送信した際に、それらの各顧客のメールを使用して謝罪で応答しつつも、顧客ごとに応答を少しパーソナライズしたい場合があります。次のセルでは、このパターンを実現するために `PromptTemplate` を作成します。

In [ ]:
# Create a prompt template that has multiple input variables
from langchain.prompts import PromptTemplate

multi_var_prompt = PromptTemplate(
    input_variables=["customerServiceManager", "customerName", "feedbackFromCustomer"], 
    template="""

Human: Create an apology email from the Service Manager {customerServiceManager} at AnyCompany to {customerName} in response to the following feedback that was received from the customer: 
<customer_feedback>
{feedbackFromCustomer}
</customer_feedback>

Assistant:"""
)

# Pass in values to the input variables
prompt = multi_var_prompt.format(customerServiceManager="Bob Smith", 
                                 customerName="John Doe", 
                                 feedbackFromCustomer="""こんにちは、Bobさん。
最近、カスタマー サポートに電話した際に経験したことに非常に失望しています。
すぐに折り返しの電話がかかってくると思っていましたが、折り返しの電話がかかってくるのに 3 日かかりました。
問題を解決するための最初の提案は間違っていました。結局、問題は 3 日後に解決しました。
提供された回答には非常に不満で、他の会社に依頼することを検討するかもしれません。
"""
     )


In [ ]:
# get number of tokens
num_tokens = chat_model.get_num_tokens(prompt)
print(f"Our prompt has {num_tokens} tokens")

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:**  エラーまたは警告が出た場合、無視して次のセルに進むことができます。

In [ ]:
#invoke
response = chat_model.invoke(prompt)

In [ ]:
# Configure a Chain to parse output
chain = StrOutputParser()
formatted_response=chain.invoke(response)
print(formatted_response)

コンテキストなしで LLM を呼び出すと、目的の結果が得られない可能性があることを学習しました。コンテキストを追加し、プロンプト テンプレートを使用して LLM からの出力を制限することで、目的の出力を正常に取得できました。

### 試してみましょう
- 特定のユースケースに合わせてプロンプトを変更し、さまざまなモデルの出力を評価します。
- トークンの長さを変えることで、サービスのレイテンシと応答性がどのように変化するかを理解します。
- さまざまなプロンプトエンジニアリングの原則を適用して、より良い出力を取得します。

### クリーンアップ

あなたはこのノートブックを完了しました。ラボの次のパートに移るには、下記を実行してください。:

- このノートブックファイルを閉じます。
- ラボセッションに戻り、**タスク 2** に進んでください。
